#  Spike PAtterns DEtection using SPADE


In [ ]:
import numpy as np
import quantities as pq
import neo
import elephant
import viziphant
np.random.seed(4542)

## introducing heterogeneous delays

the function developed in the scan + a shift in time

In [ ]:
t_stop = 10*pq.s
slope = 1.
A_sync = 0.05
shift = 0.0
A = [0]+[1.-A_sync]+[0]*8+[A_sync]
spiketrains = elephant.spike_train_generation.compound_poisson_process(
                             rate=5*pq.Hz, A=A, shift=shift*pq.ms, t_stop=10*pq.s)
                             


Add a shift:

In [ ]:
spiketrains[0].times + 4.2*pq.s

Wrap around time:

In [ ]:
np.sort(np.mod(spiketrains[0].times + 4.2*pq.s, 10*pq.s))

Sort in time:

In [ ]:
np.sort(np.mod(spiketrains[0].times + 4.2*pq.s, 10*pq.s))

https://neo.readthedocs.io/en/stable/core.html?highlight=spiketrain#example-spiketrain

In [ ]:
neo.SpikeTrain(np.sort(np.mod(spiketrains[0].times + 4.2*pq.s, 10*pq.s)), t_start=0*pq.s, t_stop=10*pq.s)



In [ ]:
for i_st in range(10):
    delay = (i_st*slope)*pq.ms
    spike_times = np.sort(np.mod(spiketrains[i_st].times + delay, t_stop))
    spiketrains[i_st] = neo.SpikeTrain(spike_times, t_start=0*pq.s, t_stop=t_stop)

In [ ]:
print(slope, 'max delay=', 10*slope, 'ms')

In [ ]:

import matplotlib.pyplot as plt
plt.figure(figsize=(15, 7))
plt.eventplot([spiketrains[i].magnitude for i in range(len(spiketrains))], linelengths=0.75, color='black')
plt.xlabel('Time (ms)', fontsize=16)
plt.ylabel('Neuron')
plt.title("Raster plot");

All in one function:

In [ ]:

def generate_and_test(A_sync=A_sync, shift=shift, winlen=1, slope=slope, t_stop=t_stop):

    A = [0]+[1.-A_sync]+[0]*8+[A_sync]
    spiketrains = elephant.spike_train_generation.compound_poisson_process(
                             rate=5*pq.Hz, A=A, shift=shift*pq.ms, t_stop=t_stop)
    for i_st in range(10):
        delay = (i_st*slope)*pq.ms
        spike_times = np.sort(np.mod(spiketrains[i_st].times + delay, t_stop))
        spiketrains[i_st] = neo.SpikeTrain(spike_times, t_start=0*pq.s, t_stop=t_stop)

    for i in range(90):
        spiketrains.append(elephant.spike_train_generation.homogeneous_poisson_process(
            rate=5*pq.Hz, t_stop=10*pq.s))
    patterns = elephant.spade.spade(
                                spiketrains=spiketrains, binsize=1*pq.ms, winlen=winlen, min_spikes=3,
                                n_surr=100, dither=5*pq.ms,
                                psr_param=[0,0,0],
                                output_format='patterns')['patterns']
    return patterns


* changing the synchrony probability to check when SPADE will begin to fail:


In [ ]:
for slope_ in np.logspace(-1, 1, 10, base=10):
    patterns = generate_and_test(slope=slope_)
    print(slope_, 'max delay=', 10*slope, 'ms: ', patterns)

In [ ]:
for slope_ in np.logspace(-1, 1, 10, base=10):
    patterns = generate_and_test(slope=slope_, winlen=10)
    print(slope_, 'max delay=', 10*slope, 'ms: ', patterns)